<a href="https://colab.research.google.com/github/Bryan-Az/Mathematics-LLM/blob/main/%5BEvaluation%2C_GGUF%2C_Quantization%5D_Mathematics_LLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the 'Storybook' Story Generation Model
This notebook is running on the TPUv2-8 GPU environment in Google Colab. The pre-trained foundation model we are using is the gated meta-llama/Meta-Llama-3-8B-Instruct, requiring authentication with HuggingFace.

## Imports and Installs

In [ ]:
!pip3 install transformers zstandard jsonlines peft wandb bitsandbytes -q
!pip3 install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install accelerate datasets sentencepiece langchain 'torch_xla[tpu] @ https://storage.googleapis.com/pytorch-xla-releases/wheels/tpuvm/torch_xla-2.6.0.dev-cp310-cp310-linux_x86_64.whl' -f https://storage.googleapis.com/libtpu-releases/index.html
!pip uninstall -y tensorflow
!pip install tensorflow-cpu -q
!git clone https://github.com/IsNoobgrammer/Pytorch-Optimizers optims

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.5/317.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
Looking in links: https://storage.googleapis.com/libtpu-releases/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 

In [ ]:
!pip uninstall -y torch_xla
!pip install cloud-tpu-client torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
!pip install --pre torch_xla[tpu] --index-url https://download.pytorch.org/whl/nightly/cu118 -f https://storage.googleapis.com/libtpu-releases/index.html

Found existing installation: torch-xla 2.6.0+git102cd48
Uninstalling torch-xla-2.6.0+git102cd48:
  Successfully uninstalled torch-xla-2.6.0+git102cd48
Looking in indexes: https://download.pytorch.org/whl/nightly/cu118
ERROR: Could not find a version that satisfies the requirement cloud-tpu-client (from versions: none)
ERROR: No matching distribution found for cloud-tpu-client
Looking in indexes: https://download.pytorch.org/whl/nightly/cu118
Looking in links: https://storage.googleapis.com/libtpu-releases/index.html
ERROR: Could not find a version that satisfies the requirement torch_xla[tpu] (from versions: none)
ERROR: No matching distribution found for torch_xla[tpu]


In [ ]:
import os
import pandas as pd
import numpy as np
import datasets
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch
import torch.nn as nn
import torch_xla.test.test_utils as test_utils
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import (
 AutoTokenizer, AutoModelForCausalLM, set_seed, DataCollatorWithPadding, AutoConfig
)

from transformers import logging as hf_logging
import torch_xla.runtime as xr

xr.use_spmd()

from torch_xla.experimental.xla_sharding import Mesh

from peft import LoraConfig, TaskType, get_peft_model
from datasets import  load_dataset, concatenate_datasets
from tqdm import tqdm

from torch.utils.data import Dataset as TorchDataset
from torch_xla.utils.checkpoint import checkpoint

try:
    !export USE_TORCH=True #If we don't do this, transformers will seemingly bork the session upon import. Really weird error.
    os.environ["PJRT_DEVICE"] = "TPU"
    os.environ.pop('TPU_PROCESS_ADDRESSES')
    os.environ.pop('CLOUD_TPU_TASK_ID')
    hf_logging.set_verbosity_error() # It can still display warnings which is a bit annoying but whatever
except:
    pass

ImportError: /usr/local/lib/python3.10/dist-packages/_XLAC.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN5torch4lazy13MetricFnValueB5cxx11Ed

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
# login with huggginface for using gated LLaMA foundational models
!huggingface-cli login --token $hf_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `generaluse` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `generaluse`


## Loading the Dataset on a Distributed Environment

In [ ]:
%%writefile spmd_util.py
import math
from dataclasses import dataclass, field
from typing import List, Optional
from collections import defaultdict
import torch
import torch.nn as nn
import re
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import LlamaConfig

# these are model architecture formatting rules specific to LLaMA
LLAMA_RULES = (
    ("model\\.embed_tokens", ("mp", "fsdp")),
    ("self_attn\\.(q_proj|k_proj|v_proj)", ("fsdp", "mp")),
    ("self_attn\\.o_proj", ("mp", "fsdp")),
    ("mlp\\.gate_proj", ("fsdp", "mp")),
    ("mlp\\.down_proj", ("mp", "fsdp")),
    ("mlp\\.up_proj", ("fsdp", "mp")),
    ("lm_head", ("fsdp", "mp")),
)

strkey2id = {
    "dp": 0, ## usefull for sharding inputs
    "fsdp": 1, ## Pytorch-Xla (2D-sharding) axis to shard data (mostly mesh shape will be (8,1)) data will be sharded 8 way
    "mp": 2 ## axis to shard model model will be sharded one way
               ## Recommened checking Pytorch-tpu/transfomers on github (xla-fork of transformers)
}

def partition_module(model, mesh, device=xm.xla_device(), verbose=False):
    partition_specs = find_rule(model)
    rule = [(k, tuple([strkey2id[x] for x in v])) for k, v in partition_specs]

    # print(rule)

    for name, module in model.named_modules():
        module.to(device)
#         print(name, module.__class__.__name__)
        if isinstance(module, (nn.Embedding, nn.Linear)):
            for rule_pattern, spec in rule:
                if re.findall(rule_pattern, name.lower())  : # and ("lora" not in name.lower()):
                    if verbose:
                        print("match", rule_pattern, name)

                    xs.mark_sharding(module.weight, mesh, spec)
                    break

Writing spmd_util.py


In [ ]:
# prompt: i want to load the datasets using a tpu

import pandas as pd
import tarfile
import tensorflow as tf


tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
tpu_spec = tpu.cluster_spec().as_dict()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

# Open the tar.gz file
tar = tarfile.open('/content/TinyStories all data.tar.gz', 'r:gz')

# List all files in the archive
files = tar.getmembers()

# Find the JSON files within the archive
json_files = [file for file in files if file.name.endswith('.json')]

# Initialize an empty list to store dataframes
dfs = []

with tpu_strategy.scope():  # Put data loading within TPU strategy scope
  # Iterate through the JSON files and load them into pandas
  for json_file in json_files:
    # Extract the file from the archive
    extracted_file = tar.extractfile(json_file)
    # Read the JSON data into a pandas dataframe
    df = pd.read_json(extracted_file)
    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
all_data_tinystories = pd.concat(dfs, ignore_index=True)

# Close the tar file
tar.close()

ValueError: Please provide a TPU Name to connect to.

In [ ]:
all_data_tinystories.head()

In [ ]:
all_data_tinystories.shape